In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import PrecisionRecallDisplay

from scipy import stats

In [84]:
#import the dataframe and get a list of titles
df = pd.read_csv('2950 Data Annotated.txt', sep='^', header = None)
df.columns = ['Link', 'Title', 'Author', 'Year', 'Authors Gender', 'Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Negative', 'Positive', 'Sadness', 'Surprise', 'Trust', 'Number of Hits']
df_trimmed = df[df['Year'] >= 1800]
df_trimmed = df_trimmed.reset_index(drop=True)
titles = df_trimmed['Title'].tolist()
df_trimmed

,Link,Title,Author,Year,Authors Gender,Anger,Anticipation,Disgust,Fear,Joy,Negative,Positive,Sadness,Surprise,Trust,Number of Hits
0,https://www.gutenberg.org/cache/epub/28561/pg2...,Josephine Makers of History,Abbot Jacob,1841,m,2761,2782,2046,3674,1889,6044,7673,2424,1082,3429,16173
1,https://www.gutenberg.org/files/4667/4667-0.txt,Seven Wives and Seven Prisons,L.A. Abbott,1870,m,1913,1683,1541,2187,1165,4011,4493,1931,717,2305,10517
2,https://www.gutenberg.org/cache/epub/67472/pg6...,How to Pick a Mate The Guide to a Happy Marriage,Clifford Rose Adams and Vance O. Packard,1946,m,3115,3094,2357,3585,2286,7045,8180,2873,1560,4503,18344
3,https://www.gutenberg.org/files/59333/59333-0.txt,Harolds Bride: A Tale,Charlotte Maria Tucker,1902,f,2443,2302,2333,3187,1824,5375,6155,2450,1329,2632,13848
4,https://www.gutenberg.org/cache/epub/61804/pg6...,Sheer Off: A Tale,Charlotte Maria Tucker,1870,f,3740,3223,3223,4303,2561,7178,9112,3647,1613,4356,19321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,https://www.gutenberg.org/cache/epub/48181/pg4...,Six Bad Husbands and Six Unhappy Wives,Ella Wheeler Wilcox,1912,f,396,435,424,467,400,948,1357,387,176,676,2789
168,https://www.gutenberg.org/cache/epub/35389/pg3...,Why Marry?,Jesse Lynch Williams,1914,m,1349,1231,1477,1529,1127,3307,3549,1350,699,1786,8158
169,https://www.gutenberg.org/cache/epub/26187/pg2...,The Gentle Art of Cooking Wives,Elizabeth Strong Worthington,1900,f,1728,1672,1541,2003,1202,3839,4674,1698,931,1855,10268
170,https://www.gutenberg.org/cache/epub/12485/pg1...,The Three Brides,Charlotte M. Yonge,1876,f,6811,6059,5693,7961,4753,14338,18640,7156,2853,8008,38602


In [52]:
#import the cleaned lexicon
lexicon_no_zeros = np.genfromtxt('new_lexicon_no_zeros.txt', delimiter=' ', dtype=None,encoding = 'UTF-8')

In [57]:
# generate scores for titles
pos_title = []
neg_title = []

for index in range(len(titles)):
    txtStr = titles[index]

    pos_score = 0
    neg_score = 0
    str_bldr = ''
    i = 0
    while i < len(txtStr):
        if txtStr[i] == '\\':
            i = i+2            
        elif not (str(txtStr[i]) in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ "):
            i = i+1
        else:
            str_bldr = str_bldr + txtStr[i]
            i=i+1

    exp = str_bldr.lower().split(" ")

    for i in range(len(exp)):
        j=0
        terminate = 0
        while (j<6453 and terminate == 0): #6453 is the length of the cleaned lexicon
            if (not ('mar' in exp[i]) and not ('bride' in exp[i]) and not ('groom' in exp[i]) and (lexicon_no_zeros[j][0] == exp[i]) or (lexicon_no_zeros[j][0]+'s' == exp[i])):
                pos_score += lexicon_no_zeros[j][7]
                neg_score += lexicon_no_zeros[j][6]
                terminate =1
            j+=1

    pos_title.append(pos_score)
    neg_title.append(neg_score)

In [76]:
data = {'Pos_Title':pos_title, 'Neg_Title':neg_title}
new_predictors = pd.DataFrame(data)


(13, 2)

In [85]:
df_trimmed['Pos_Title'] = pos_title
df_trimmed['Neg_Title'] = neg_title


,Link,Title,Author,Year,Authors Gender,Anger,Anticipation,Disgust,Fear,Joy,Negative,Positive,Sadness,Surprise,Trust,Number of Hits,Pos_Title,Neg_Title
0,https://www.gutenberg.org/cache/epub/28561/pg2...,Josephine Makers of History,Abbot Jacob,1841,m,2761,2782,2046,3674,1889,6044,7673,2424,1082,3429,16173,0,0
1,https://www.gutenberg.org/files/4667/4667-0.txt,Seven Wives and Seven Prisons,L.A. Abbott,1870,m,1913,1683,1541,2187,1165,4011,4493,1931,717,2305,10517,0,1
2,https://www.gutenberg.org/cache/epub/67472/pg6...,How to Pick a Mate The Guide to a Happy Marriage,Clifford Rose Adams and Vance O. Packard,1946,m,3115,3094,2357,3585,2286,7045,8180,2873,1560,4503,18344,4,0
3,https://www.gutenberg.org/files/59333/59333-0.txt,Harolds Bride: A Tale,Charlotte Maria Tucker,1902,f,2443,2302,2333,3187,1824,5375,6155,2450,1329,2632,13848,1,0
4,https://www.gutenberg.org/cache/epub/61804/pg6...,Sheer Off: A Tale,Charlotte Maria Tucker,1870,f,3740,3223,3223,4303,2561,7178,9112,3647,1613,4356,19321,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,https://www.gutenberg.org/cache/epub/48181/pg4...,Six Bad Husbands and Six Unhappy Wives,Ella Wheeler Wilcox,1912,f,396,435,424,467,400,948,1357,387,176,676,2789,0,2
168,https://www.gutenberg.org/cache/epub/35389/pg3...,Why Marry?,Jesse Lynch Williams,1914,m,1349,1231,1477,1529,1127,3307,3549,1350,699,1786,8158,0,0
169,https://www.gutenberg.org/cache/epub/26187/pg2...,The Gentle Art of Cooking Wives,Elizabeth Strong Worthington,1900,f,1728,1672,1541,2003,1202,3839,4674,1698,931,1855,10268,1,0
170,https://www.gutenberg.org/cache/epub/12485/pg1...,The Three Brides,Charlotte M. Yonge,1876,f,6811,6059,5693,7961,4753,14338,18640,7156,2853,8008,38602,1,0


In [148]:
# English and American Writers make up the bulk of this list. 
# As a matter of time, I cannot match every writer with the birth rate of their nation within that time span
# I calcuate the relevant birth rate by averaging the US, and English birth rate scores from 1-5 years
# as English and American writers make up the bulk of the authors
# from: https://www.gapminder.org/data/documentation/

fertility_df = pd.read_csv('Fertility Dataset Extracted.csv', sep=',')
fertility = (fertility_df.T)[2]
fert_df = pd.DataFrame(fertility)
fert_df.columns = ['US_UK_Avg']
fert_df.loc['1860',:][0] #gets the rate in 1860

5.07

In [146]:
def get_5yr (yr):
    if(yr == 1800):
        return(fert_df.loc['1800',:][0])
    elif (yr == 1801):
        return(fert_df.loc['1800',:][0]+fert_df.loc['1801',:][0])
    elif (yr == 1802):
        return(fert_df.loc['1800',:][0]+fert_df.loc['1801',:][0]+fert_df.loc['1802',:][0])
    elif (yr == 1803):
        return(fert_df.loc['1800',:][0]+fert_df.loc['1801',:][0]+fert_df.loc['1802',:][0]+fert_df.loc['1803',:][0])
    ans_x5 = fert_df.loc[str(yr-0),:][0] + fert_df.loc[str(yr-1),:][0] + fert_df.loc[str(yr-2),:][0] + fert_df.loc[str(yr-3),:][0] + fert_df.loc[str(yr-4),:][0]
    return(ans_x5/5)

In [149]:
fert_5yr = []
for yr in df_trimmed['Year']:
    fert_5yr.append(get_5yr(yr))
df_trimmed['5yr_fertility'] = fert_5yr
df_trimmed.head(1)

,Link,Title,Author,Year,Authors Gender,Anger,Anticipation,Disgust,Fear,Joy,Negative,Positive,Sadness,Surprise,Trust,Number of Hits,Pos_Title,Neg_Title,5yr_fertility
0,https://www.gutenberg.org/cache/epub/28561/pg2...,Josephine Makers of History,Abbot Jacob,1841,m,2761,2782,2046,3674,1889,6044,7673,2424,1082,3429,16173,0,0,5.527


In [211]:
#War where 0.01% or more of population dies
#Source is wikipedia

us_war = [
    ['American Civil War', 1861, 1865, 655000],
    ['World War 2', 1941, 1945, 405399],
    ['World War 1', 1917, 1918, 116516],
    ['Vietnam War', 1961, 1975, 58209],
    ['Korean War', 1950, 1953, 36574],
    ['War of 1812', 1812, 1815, 15000],
    ['Mexican American War', 1846, 1848, 13283]
]
uk_war = [
    ['World War 1', 1914, 1918, 994858],
    ['World War 2', 1939, 1945, 450900],
    ['Second Boer War', 1899, 1902, 22000],
    ['Crimean War', 1854, 1856, 22182],
    ['First Anglo Bermese War', 1824, 1826, 15000],
    ['Indian Rebellion of 1857', 1857, 1858, 11021],
    ['Napoleonic Wars', 1804, 1815, 100000],
    ['French Revolutionary Wars', 1792, 1802, 30000]
]

#Rank 2 wars are masive bloody wars that impact the home front greatly
#Very heavy fighting from 1792 - 1802 and 1804 - 1815. (This period is classified as rank 2 war)
#WW1 and WW2 have their names for a reason 1914-1918 and 1939 - 1945. (This period is classified as a rank 2 war)
#American Civil War is rank 2 -> huge number of deaths as percent of population & war was directly on the home front

# https://en.wikipedia.org/wiki/Napoleonic_Wars_casualties#cite_note-White_2014-15
# https://en.wikipedia.org/wiki/French_Revolutionary_Wars
# https://en.wikipedia.org/wiki/United_States_military_casualties_of_war
# https://en.wikipedia.org/wiki/United_Kingdom_casualties_of_war

#1961-1975 -> rank 1 war
#1950-1953 -> rank 1 war
#1939-1945 -> rank 2 war
#1914-1918 -> rank 2 war
#1899-1902 -> rank 1 war
#1861-1865 -> rank 2 war
#1854-1858 -> rank 1 war
#1824-1826 -> rank 1 war
#1804-1815 -> rank 2 war
#1792-1802 -> rank 2 war

conflict_yr = pd.DataFrame({'Year':range(1800,1975)})
tier = np.zeros(175)
tier[0:2+1] = 2
tier[4:15+1] = 2
tier[61:65+1] = 2
tier[114:118+1] = 2
tier[139:145+1] = 2
tier[24:26+1] = 1
tier[54:58+1] = 1
tier[99:102+1] = 1
tier[150:153+1] = 1
tier[161:175+1] = 1
conflict_yr['Tier']=tier
conflict_yr

,Year,Tier
0,1800,2.0
1,1801,2.0
2,1802,2.0
3,1803,0.0
4,1804,2.0
...,...,...
170,1970,1.0
171,1971,1.0
172,1972,1.0
173,1973,1.0


In [212]:
# df_trimmed['War_Tier'] = conflict_yr.loc[(df_trimmed['Year']-1800),1]
# df_trimmed

yr_lst = df_trimmed['Year'].tolist()
tier_list = []

t_lst = conflict_yr['Tier'].tolist()

for i in yr_lst:
    inp = t_lst[i-1800]
    tier_list.append(inp)
    
df_trimmed['War_Tier'] = tier_list

In [214]:
#Lets add in one last predictor - Depression and Recession in the United states
#depression is 2
#recession is 1

rec = pd.DataFrame({'Year':range(1800,1975)})
tier = np.zeros(175)
tier[2:4+1] = 1.0
tier[7:10+1] = 2
tier[12] = 1
tier[15:21+1] = 1
tier[22:23+1] = 1
tier[25:26+1] = 1
tier[28:29+1] = 1
tier[33:34+1] = 1
tier[36:43+1] = 1
tier[45:48+1] = 1
tier[53:54+1] = 1
tier[57] = 1
tier[60:61+1] = 1
tier[65:67+1] = 1
tier[69:70+1] = 1
tier[73:79+1] = 2
tier[82:85+1] = 1
tier[87:88+1] = 1
tier[90:91+1] = 1
tier[93:94+1] = 1
tier[95:97+1] = 1
tier[99:100+1] = 1
tier[102:104+1] = 1
tier[107:108+1] = 1
tier[110:111+1] = 1
tier[113:114+1] = 1
tier[118:119+1] = 1
tier[120:121+1] = 2
tier[123:124+1] = 1
tier[126:127+1] = 1
tier[129:133+1] = 2
tier[137:138+1] = 1
tier[145] = 1
tier[148:149+1] = 1
tier[153:154+1] = 1
tier[157:158+1] = 1
tier[160:161+1] = 1
tier[169:170+1] = 1
tier[173:175+1] = 1
rec['Tier']=tier
for i in range(len(tier)):
    print(str(i+1800) + ' ' + str(tier[i]))

1800 0.0
1801 0.0
1802 1.0
1803 1.0
1804 1.0
1805 0.0
1806 0.0
1807 2.0
1808 2.0
1809 2.0
1810 2.0
1811 0.0
1812 1.0
1813 0.0
1814 0.0
1815 1.0
1816 1.0
1817 1.0
1818 1.0
1819 1.0
1820 1.0
1821 1.0
1822 1.0
1823 1.0
1824 0.0
1825 1.0
1826 1.0
1827 0.0
1828 1.0
1829 1.0
1830 0.0
1831 0.0
1832 0.0
1833 1.0
1834 1.0
1835 0.0
1836 1.0
1837 1.0
1838 1.0
1839 1.0
1840 1.0
1841 1.0
1842 1.0
1843 1.0
1844 0.0
1845 1.0
1846 1.0
1847 1.0
1848 1.0
1849 0.0
1850 0.0
1851 0.0
1852 0.0
1853 1.0
1854 1.0
1855 0.0
1856 0.0
1857 1.0
1858 0.0
1859 0.0
1860 1.0
1861 1.0
1862 0.0
1863 0.0
1864 0.0
1865 1.0
1866 1.0
1867 1.0
1868 0.0
1869 1.0
1870 1.0
1871 0.0
1872 0.0
1873 2.0
1874 2.0
1875 2.0
1876 2.0
1877 2.0
1878 2.0
1879 2.0
1880 0.0
1881 0.0
1882 1.0
1883 1.0
1884 1.0
1885 1.0
1886 0.0
1887 1.0
1888 1.0
1889 0.0
1890 1.0
1891 1.0
1892 0.0
1893 1.0
1894 1.0
1895 1.0
1896 1.0
1897 1.0
1898 0.0
1899 1.0
1900 1.0
1901 0.0
1902 1.0
1903 1.0
1904 1.0
1905 0.0
1906 0.0
1907 1.0
1908 1.0
1909 0.0
1910 1.0
1

In [219]:
#The books were likely written in the 3 years before the publication date. 
#We average the year plus, the 3 years before it to estimate the total recession score

def get_rec_3yr(yr):
    if(yr == 1800):
        return 0.75
    elif (yr == 1801):
        return 0.5
    elif (yr == 1802):
        return 0.5
    ans_x4 = tier[yr-1800] + tier[yr-1801] + tier[yr-1802] + tier[yr-1803]
    return(ans_x4/4)

In [223]:
yr_lst = df_trimmed['Year'].tolist()
dep_list = []

for i in yr_lst:
    inp = get_rec_3yr(i)
    dep_list.append(inp)
    
df_trimmed['Recession'] = dep_list
df_final = df_trimmed
df_final.head(2)

,Link,Title,Author,Year,Authors Gender,Anger,Anticipation,Disgust,Fear,Joy,...,Positive,Sadness,Surprise,Trust,Number of Hits,Pos_Title,Neg_Title,5yr_fertility,War_Tier,Recession
0,https://www.gutenberg.org/cache/epub/28561/pg2...,Josephine Makers of History,Abbot Jacob,1841,m,2761,2782,2046,3674,1889,...,7673,2424,1082,3429,16173,0,0,5.527,0.0,1.00
1,https://www.gutenberg.org/files/4667/4667-0.txt,Seven Wives and Seven Prisons,L.A. Abbott,1870,m,1913,1683,1541,2187,1165,...,4493,1931,717,2305,10517,0,1,4.988,0.0,0.75


In [224]:
np.savetxt("Final Data.txt", df_final, fmt="%s", delimiter = '^')